Tying Weights of Symmetric Autoencoders

In [1]:
import tensorflow as tf
from tensorflow import keras

In [4]:
# custome layer
class DenseTransposed(keras.layers.Layer):
    def __init__(self, dense, activation= None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)

    def build(self, batch_input_shape):
        kernel = self.dense.weights[0]
        output_dim_transposed = kernel.shape[0]
        self.biases = self.add_weight(name= "bias", initializer= "zeros", shape= [output_dim_transposed])
        super().build(batch_input_shape)

    def call(self, inputs):
        kernel = self.dense.weights[0]
        z = tf.matmul(inputs, kernel, transpose_b= True) # shape(weights[0]) = [output_dim, input_dim] (why we transpose)
        return self.activation(z + self.biases)

In [5]:
# tied autoencoder
dense_1 = keras.layers.Dense(100, activation= "selu", use_bias= False)
dense_2 = keras.layers.Dense(30, activation= "selu", use_bias= False)

tied_encoder = keras.models.Sequential([
    keras.layers.Input(shape= [28, 28]),
    keras.layers.Flatten(),
    dense_1,
    dense_2
])

tied_decoder = keras.models.Sequential([
    DenseTransposed(dense_2, activation= "selu"),
    DenseTransposed(dense_1, activation= "sigmoid"),
    keras.layers.Reshape([28, 28])
])

tied_autoencoder = keras.models.Sequential([tied_encoder, tied_decoder])